# The SVM vs. Logistic Regression Showdown

In this lab, you will practice working with non-linear kernels combined with logistic regression and SVM classifiers. The goal is to compare these commonly used techniques. Which comes out on top in terms of accuracy? Runtime? Is there much of a difference at all? Is the dominance of the SVM classifier in machine learning pedagogy justified?

## Loading the Data

First, we load all the packages we'll need.

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import pairwise_kernels
import scipy
from sklearn import svm, linear_model
from sklearn.model_selection import GridSearchCV
import time

Again we download the data from the Tensorflow package, which you will need to install.  You can get the data from other sources as well.

In the Tensorflow dataset, the training and test data are represented as arrays:

     Xtr.shape = 60000 x 28 x 28
     Xts.shape = 10000 x 28 x 28
     
The test data consists of `60000` images of size `28 x 28` pixels; the test data consists of `10000` images.

In [3]:
!pip install tensorflow

import tensorflow as tf

(Xtr_raw,ytr),(Xts_raw,yts) = tf.keras.datasets.mnist.load_data()

print('Xtr shape: %s' % str(Xtr_raw.shape))
print('Xts shape: %s' % str(Xts_raw.shape))

ntr = Xtr_raw.shape[0]
nts = Xts_raw.shape[0]
nrow = Xtr_raw.shape[1]
ncol = Xtr_raw.shape[2]

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/e4/14/d795bb156f8cc10eb1dcfe1332b7dbb8405b634688980aa9be8f885cc888/tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.16.1 from https://files.pythonhosted.org/packages/e0/36/6278e4e7e69a90c00e0f82944d8f2713dd85a69d1add455d9e50446837ab/tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2db

Each pixel value is from `[0,255]`.  For this lab, it will be convenient to recale the value to -1 to 1 and reshape it as a `ntr x npix` and `nts x npix`.

In [4]:
npix = nrow*ncol
Xtr = Xtr_raw.reshape((ntr,npix))
print(Xtr[1,:])
Xtr = (Xtr/255 - .5)
print(Xtr[1,:])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0  51 159 253 159  50   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0  48 238 252 252 252 237   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0  54 227 253 252 239 233 252  57   6   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  10  60 224 252 253 252 202  84 252
 253 122   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0 163 252 252 252 253 252 252  96 189 253 167   

In [5]:
npix = nrow*ncol
Xtr = (Xtr_raw/255 - 0.5)
Xtr = Xtr.reshape((ntr,npix))

Xts = (Xts_raw/255 - 0.5)
Xts = Xts.reshape((nts,npix))

In [6]:
print(Xtr)

[[-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 ...
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]]


For this lab we're only going to use a fraction of the MNIST data -- otherwise our models will take too much time and memory to run. Using only part of the training data will of course lead to worse results. Given enough computational resources and time, we would ideally be running on the full data set. The follow code creates a new test and train set, with 10000 examples for train and 5000 for test.

In [137]:
ntr1 = 10000
nts1 = 5000
Iperm = np.random.permutation(ntr1)
Xtr1 = Xtr[Iperm[:ntr1],:]
ytr1 = ytr[Iperm[:ntr1]]
Iperm = np.random.permutation(nts1)
Xts1 = Xts[Iperm[:nts1],:]
yts1 = yts[Iperm[:nts1]]

In [138]:
print(Xts1)

[[-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 ...
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]]


## Problem set up and establishing a baseline

To simplify the problem (and speed things up) we're also going to restrict to binary classification. In particular, let's try to design classifier a that separates the 8's from all other digits.

Create binary 0/1 label vectors `ytr8` and `yts8` which are 1 wherever `ytr1` and `yts1` equal 8, and 0 everywhere else.

In [139]:
# TODO
ytr8 = (ytr1 == 8).astype(int)
yts8 = (yts1 == 8).astype(int)

In [141]:
ytr8

array([0, 0, 0, ..., 0, 0, 0])

Most of the digits in the test dataset aren't equal to 8. So if we simply guess 0 for every image in `Xts`, we might expect to get classification accuracy around 90%. Our goal should be to significantly beat this **baseline**.

Formally, write a few lines of code to check what test error would be achieved by the all zeros classifier.

In [142]:
# TODO
predict = np.zeros(len(Xts))
acc = np.mean(yts8 == 0)
print('Accuaracy = {0:f}'.format(acc))

Accuaracy = 0.902200


As a second baseline, let's see how we do with standard (non-kernel) logistic regression. As in the MNIST demo, you can use `scikit-learn`'s built in function `linear_model.LogisticRegression` to fit the model and compute the accuracy. Use no regularization and the `lbfgs` solver. You should acheive an improvement to around 93-95%.

In [278]:
# TODO
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(C=1e5, solver='lbfgs', max_iter=100000)
model.fit(Xtr1, ytr8)


C:\Users\Sky\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=100000.0, max_iter=100000)

In [285]:
predictions = model.predict(Xts1)
acc = accuracy_score(yts8, predictions)
print('Logistic Regression Accuaracy = {0:f}'.format(acc))

Logistic Regression Accuaracy = 0.929600


## Kernel Logistic Regression

To improve on this baseline performance, let's try using the logistic regression classifier with a *non-linear* kernel. Recall from class that any non-linear kernel similarity function $k(\vec{w},\vec{z})$ is equal to $\phi(\vec{w})^T\phi(\vec{z})$ for some feature transformation $\phi$. However, we typically do not need to compute this feature tranformation explicitly: instead we can work directly with the kernel gram matrix $K \in \mathbb{R}^{n\times n}$. Recall that $K_{i,j} = k(\vec{x}_i,\vec{x}_j)$ where $\vec{x}_i$ is the $i^\text{th}$ training data point.

For this lab we will be using the radial basis function kernel. For a given scaling factor $\gamma$ this kernel is defined as:
$$
k(\vec{w},\vec{z}) = e^{-\gamma\|\vec{w}-\vec{z}\|_2^2}
$$

In [59]:
def rbf_kernel(w,z,gamma):
    d = w - z
    return np.exp(-gamma*np.sum(d*d))

Construct the kernel matrix `K1` for `Xtr1` with `gamma = .05`.

In [145]:
from scipy.spatial.distance import cdist
def compute_kernel_matrix(X, gamma):
    # Compute the squared Euclidean distance matrix
    sq_dists = cdist(X, X, 'sqeuclidean')
    # Compute the kernel matrix
    K = np.exp(-gamma * sq_dists)
    return K

In [146]:
# TODO
gamma = .05
K1 = compute_kernel_matrix(Xtr1, gamma)

In [147]:
print(K1)

[[1.         0.02510458 0.00338946 ... 0.01088088 0.00380884 0.009766  ]
 [0.02510458 1.         0.019551   ... 0.00734931 0.00976997 0.01208535]
 [0.00338946 0.019551   1.         ... 0.00378603 0.01146406 0.00212038]
 ...
 [0.01088088 0.00734931 0.00378603 ... 1.         0.00744453 0.01824785]
 [0.00380884 0.00976997 0.01146406 ... 0.00744453 1.         0.00549742]
 [0.009766   0.01208535 0.00212038 ... 0.01824785 0.00549742 1.        ]]


If you used a for loop (which is fine) your code might take several minutes to run! Part of the issue is that Python won't know to properly parallize your for loop. For this reason, when constructing kernel matrices it is often faster to us a built-in, carefully optimized function with explicit parallelization. Scikit learn provides such a function through their `metrics` library.

Referring to the documentation here
https://scikit-learn.org/stable/modules/metrics.html#metrics, use this built in function to recreate the same kernel matrix you did above. Store the result at `K`.

In [148]:
# TODO
from sklearn.metrics.pairwise import rbf_kernel
K = rbf_kernel(Xtr1, Xtr1, gamma=gamma)

In [260]:
K

array([[1.        , 0.02510458, 0.00338946, ..., 0.01088088, 0.00380884,
        0.009766  ],
       [0.02510458, 1.        , 0.019551  , ..., 0.00734931, 0.00976997,
        0.01208535],
       [0.00338946, 0.019551  , 1.        , ..., 0.00378603, 0.01146406,
        0.00212038],
       ...,
       [0.01088088, 0.00734931, 0.00378603, ..., 1.        , 0.00744453,
        0.01824785],
       [0.00380884, 0.00976997, 0.01146406, ..., 0.00744453, 1.        ,
        0.00549742],
       [0.009766  , 0.01208535, 0.00212038, ..., 0.01824785, 0.00549742,
        1.        ]])

Check that you used the function correctly by writing code to confirm that `K = K1`, or at least that the two are equal up to very small differences (which could arise due to numerical precision issues). Try to do this **without a for loop** for full credit. You will get partial credit if you use a for loop.

In [149]:
# TODO
are_close = np.allclose(K, K1, atol=1e-10)
print(are_close)

True


When using a non-linear kernel, it is important to check that you have chosen reasonable parameters (in our case the only parameter is `gamma`). We typically do not want $k(\vec{x}_i,\vec{x}_j)$ to be either negligably small, or very large for all $i\neq j$ in our data set, or we won't be able to learn anything. For the RBF kernel this means that, for any $\vec{x}_i$ we don't want $k(\vec{x}_i,\vec{x}_j)$ very close to 1 (e.g. .9999) for all $j$, or very close to $0$ (e.g. 1e-8) for all $j$.

Let's just check that we're in good shape for the first data vector $\vec{x}_0$. Do so by printing out the 10 largest and 10 smallest values of $k(\vec{x}_0,\vec{x}_j)$ for $j\neq 0$. Note that we always have $k(\vec{x}_0,\vec{x}_0) = 1$ for the RBF kernel.

In [150]:
# TODO
kernel_values = K[0, 1:]

# Find the 10 largest values excluding the self-comparison
largest = np.partition(kernel_values, -10)[-10:]
smallest = np.partition(kernel_values, 10)[:10]

print('Maximum similarities: \n', np.sort(largest)[::-1])
print('Minimum similarities: \n', np.sort(smallest))

Maximum similarities: 
 [0.16667669 0.10317341 0.08614872 0.08451159 0.08234248 0.07835038
 0.07764239 0.07746367 0.07684656 0.07236095]
Minimum similarities: 
 [2.60692441e-05 3.19661095e-05 3.41648853e-05 3.67479575e-05
 3.96780043e-05 3.98211699e-05 4.25023671e-05 4.27136539e-05
 4.77027215e-05 5.05935594e-05]


### Implementation
Maybe surprisingly Scikit learn does not have an implementation for kernel logistic regression. So we have to implement our own!

Write a function function `log_fit` that minimizes the $\ell_2$-regularized logisitic regression loss:
$$
L(\boldsymbol{\alpha}) =\sum_{i=1}^n (1-y_i)(\phi(\mathbf{x}_i)^T\phi(\mathbf{X})^T\vec{\alpha}) - \log(h(\phi(\mathbf{x}_i)^T\phi(\mathbf{X})^T\boldsymbol{\alpha}) + \lambda \|\phi(\mathbf{X})^T\boldsymbol{\alpha}\|_2^2.
$$
As input it takes an $n\times n$ kernel matrix $K$ for the training data, an $n$ length vector `y` of binary class labels, and regularization parameter `lamb`.

To implement your function you can either use your own implmentation of gradient descent or used a built in minimizer from `scipy.optimize.minimize`. I recommend the later approach: as we saw in the last lab, gradient descent can converge slowly for this objective.

In [324]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_loss(alpha, K, y, lamb):
    linear_combination = K @ alpha
    predictions = sigmoid(linear_combination)
    # Regular logistic loss
    logistic_loss = np.sum((1 - y) * linear_combination - np.log(predictions + 1e-10))
    # Regularization term
    regularization = lamb * (alpha.T @ K @ alpha)
    return logistic_loss + regularization

def logistic_loss_grad(alpha, K, y, lamb):
    predictions = sigmoid(K @ alpha)
    grad_loss = K.T @ (predictions - y)
    grad_reg = 2 * lamb * (K @ alpha)
    return grad_loss + grad_reg

from scipy.optimize import minimize

def log_fit(K, y, lamb):
    n = len(y)  # Number of training samples
    alpha_init = np.zeros(n)  # Initial guess for alpha

    # Minimization process
    result = minimize(
        fun=logistic_loss,       # The loss function to minimize
        x0=alpha_init,           # Initial guess
        args=(K, y, lamb),       # Additional arguments for the loss function
        method='L-BFGS-B',       # Optimization method
        jac=logistic_loss_grad,   # Gradient of the loss function
        options={'maxls': 50}
    )

    if result.success:
        return result.x  # Return the optimized alpha
    else:
        raise ValueError("Optimization failed: " + result.message)
        
alpha_opt = log_fit(K, ytr8,0)
print("Optimized alpha:", alpha_opt)

Optimized alpha: [-0.40934477 -0.30785323 -1.05335576 ... -0.19968716 -0.15283513
  0.07462778]


Use the `log_fit` function defined above to find parameters `alpha` for the kernel logistic regression model using `lamb = 0` and `K` as constructed above (with `gamma = .05`).

Suppose we have a test dataset with $m$ examples $\vec{w}_1,\ldots, \vec{w}_m$. Once we obtain a coefficient vector $\alpha$, making predictions for any $\vec{w}_j$ in the test set requires computing:
$$
{y}_{j} = \sum_{i=1}^n \alpha_i \cdot k(\vec{w}_{j}, \vec{x}_i).
$$
where $\vec{x}_1, \ldots \vec{x}_n$ are our training data vectors. We classify $\vec{w}_{j}$ in class 0 if ${y}_{j} \leq 0$ and in class 1 if ${y}_{j} > 0$.

This computation can be rewritten in matrix form as follows:
$$
\vec{y}_{test} = K_{test}\vec{\alpha},
$$
where $\vec{y}_{text}$ is an $m$ length vector and $K_{test}$ is a $m\times n$ matrix whose $(j,i)$ entry is equal to $k(\vec{w}_{j}, \vec{x}_i)$. We classify $\vec{w}_{j}$ in class 0 if $\vec{y}_{test}[j] \leq 0$ and in class 1 if $\vec{y}_{test}[j] > 0$.


Use the `pairwise_kernels` function to construct $K_{test}$. Then make predictions for the test set and evaluate the accuracy of our kernel logistic regression classifier. You should see a pretty substantial lift in accuracy to around $97\%$

In [326]:
# TODO
K_train = rbf_kernel(Xtr1, Xtr1, gamma=0.05)

# Minimize the logistic loss function to get alpha coefficients
alpha_opt = log_fit(K_train,ytr8,0)
        

# Compute the test kernel matrix
K_test = rbf_kernel(Xts1, Xtr1, 0.05)

# Make predictions for the test set
y_test_pred = np.dot(K_test, alpha_opt)
yhat = ( y_test_pred >= 0).astype(int)
acc = np.mean(yhat == yts8)

print("Test accuracy = %f" % acc)

Test accuracy = 0.970400


## Kernel Support Vector Machine

The goal of this lab is to compare Kernel Logistic Regression to Kernel Support Vector machines. Following `demo_mnist_svm.ipynb` create and train an SVM classifier on `Xtr1` and `ytr8` using an RBF kernel with `gamma = .05` (the same value we used for logistic regression aboe). Use margin parameter `C = 10`.

In [154]:
# TODO
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
# Create the SVM classifier with RBF kernel
svm_classifier = SVC(kernel='rbf', gamma=0.05, C=10)

# Train the classifier on the training data
svm_classifier.fit(Xtr1, ytr8)

SVC(C=10, gamma=0.05)

Calculate and print the accuracy of the SVM classifier. You should obtain a similar result as for logistic regression: something close to $97\%$ accuracy.

In [155]:
# TODO
ysvm = svm_classifier.predict(Xts1)
acc = np.mean(ysvm == yts8)
print("Test accuracy = %f" % acc)

Test accuracy = 0.980200


## The Showdown

Both SVM classifiers and kernel logisitic regression require tuning parameters to obtain the best possible result. In our setting we will stick with an RBF kernel (although this could be tuned). So we only consider tuning the kernel width parameter `gamma`, as well as the regularization parameter `lamb` for logistic regression, and the margin parameter `C` for SVM. We will choose parameters using for-loops and train-test cross validation.

Train a logistic regression classifier with **all combinations** of the parameters included below in vectors `gamma` and `lamb`. For each setting of parameters, compute and print:
* the test error obtained
* the total runtime of classification in seconds (including training time and prediction time)

For computing runtime you might want to use the `time()` function from the `time` library, which we already imported ealier.

In [325]:
gammas = [.1, .05,.02,.01,.005]
lambs = [0,1e-6,1e-4,1e-2]
for gamma in gammas:
    for lamb in lambs:
        start_time = time.time()
        
        K_train = rbf_kernel(Xtr1, Xtr1, gamma=gamma)

        # Minimize the logistic loss function to get alpha coefficients
        alpha_opt = log_fit(K_train,ytr8,lamb)
        

        # Compute the test kernel matrix
        K_test = rbf_kernel(Xts1, Xtr1, gamma)

        # Make predictions for the test set
        y_test_pred = np.dot(K_test, alpha_opt)
        yhat = ( y_test_pred >= 0).astype(int)
        acc = np.mean(yhat == yts8)
        test_error = 1 - acc

        end_time = time.time()
        runtime = end_time - start_time

        # Print the results
        print(f"Gamma: {gamma}, Lambda: {lamb}, Test Error: {test_error:.4f}, Accuracy: {acc:.4f}, Total Runtime: {runtime:.4f} seconds")

Gamma: 0.1, Lambda: 0, Test Error: 0.0392, Accuracy: 0.9608, Total Runtime: 3.3166 seconds
Gamma: 0.1, Lambda: 1e-06, Test Error: 0.0392, Accuracy: 0.9608, Total Runtime: 3.0925 seconds
Gamma: 0.1, Lambda: 0.0001, Test Error: 0.0392, Accuracy: 0.9608, Total Runtime: 3.0490 seconds
Gamma: 0.1, Lambda: 0.01, Test Error: 0.0566, Accuracy: 0.9434, Total Runtime: 2.9837 seconds
Gamma: 0.05, Lambda: 0, Test Error: 0.0296, Accuracy: 0.9704, Total Runtime: 3.7466 seconds
Gamma: 0.05, Lambda: 1e-06, Test Error: 0.0296, Accuracy: 0.9704, Total Runtime: 3.7615 seconds
Gamma: 0.05, Lambda: 0.0001, Test Error: 0.0296, Accuracy: 0.9704, Total Runtime: 3.9005 seconds
Gamma: 0.05, Lambda: 0.01, Test Error: 0.0258, Accuracy: 0.9742, Total Runtime: 24.9651 seconds
Gamma: 0.02, Lambda: 0, Test Error: 0.0228, Accuracy: 0.9772, Total Runtime: 8.3309 seconds
Gamma: 0.02, Lambda: 1e-06, Test Error: 0.0232, Accuracy: 0.9768, Total Runtime: 7.9402 seconds
Gamma: 0.02, Lambda: 0.0001, Test Error: 0.0254, Accura

TODO: What was the best test error achieved, and what setting of parameters achieved this error? Was the kernel logistic regression classifier more sensitive to changes in `gamma` or `lamb`? Discuss in 1-3 short sentences below.

### The best test error achieved was 0.0172, obtained with Gamma: 0.02 and Lambda: 0.01. This setting provided the highest accuracy (0.9828) within the explored parameter grid.

Regarding sensitivity, the kernel logistic regression classifier appears to be more sensitive to changes in gamma than lambda since changes in gamma lead to more significant fluctuations in performance compared to changes in lambda. It's also noteworthy that for the same gamma value, different lambda values do not radically change the test error, indicating a stronger influence of the kernel parameter on the model performance.

Now let's do the same thing for the kernel Support Vector Classifier. Train an SVM classifier with **all combinations** of the parameters included below in vectors `gamma` and `C`. For each setting of parameters, compute:
* the test error obtained
* the total runtime of classification in seconds (including training time and prediction time)

In [157]:
import time
gammas = [.1, .05,.02,.01,.005]
Cs = [.01,.1,1,10]

In [159]:
results = []

for gamma in gammas:
    for C in Cs:
        start_time = time.time()  # Start timing
        
        # Create and train the SVM classifier
        classifier = SVC(kernel='rbf', gamma=gamma, C=C)
        classifier.fit(Xtr1, ytr8)  # Training the classifier

        # Predict on the test set
        y_pred = classifier.predict(Xts1)

        # Calculate the test error
        accuracy = accuracy_score(yts8, y_pred)
        test_error = 1 - accuracy
        
        end_time = time.time()  # End timing
        runtime = end_time - start_time  # Calculate runtime
        
        # Store results
        results.append({
            'gamma': gamma,
            'C': C,
            'test_error': test_error,
            'runtime': runtime,
            'acc': accuracy
        })

        # Optionally print results
        print(f"Gamma: {gamma}, C: {C}, Test Error: {test_error:.4f}, Accuracy: {accuracy:.4f}, Runtime: {runtime:.2f} seconds")


Gamma: 0.1, C: 0.01, Test Error: 0.0978, Accuracy: 0.9022, Runtime: 68.49 seconds
Gamma: 0.1, C: 0.1, Test Error: 0.0978, Accuracy: 0.9022, Runtime: 106.47 seconds
Gamma: 0.1, C: 1, Test Error: 0.0718, Accuracy: 0.9282, Runtime: 121.73 seconds
Gamma: 0.1, C: 10, Test Error: 0.0658, Accuracy: 0.9342, Runtime: 148.04 seconds
Gamma: 0.05, C: 0.01, Test Error: 0.0978, Accuracy: 0.9022, Runtime: 22.82 seconds
Gamma: 0.05, C: 0.1, Test Error: 0.0772, Accuracy: 0.9228, Runtime: 25.26 seconds
Gamma: 0.05, C: 1, Test Error: 0.0226, Accuracy: 0.9774, Runtime: 29.44 seconds
Gamma: 0.05, C: 10, Test Error: 0.0198, Accuracy: 0.9802, Runtime: 30.22 seconds
Gamma: 0.02, C: 0.01, Test Error: 0.0978, Accuracy: 0.9022, Runtime: 16.12 seconds
Gamma: 0.02, C: 0.1, Test Error: 0.0488, Accuracy: 0.9512, Runtime: 14.48 seconds
Gamma: 0.02, C: 1, Test Error: 0.0180, Accuracy: 0.9820, Runtime: 11.02 seconds
Gamma: 0.02, C: 10, Test Error: 0.0142, Accuracy: 0.9858, Runtime: 11.47 seconds
Gamma: 0.01, C: 0.01, T

TODO: What was the best test error achieved, and what setting of parameters achieved this error? Which performed better in terms of accuracy, the SVM or logisitic regression classifier? How about in terms of runtime?

### Best Param Combo for SVM is: Gamma: 0.01, C: 10, Test Error: 0.0130, Accuracy: 0.9870, Runtime: 8.81 seconds
In terms of accuracy, the SVM classifier performed "slightly" better than the kernel logistic regression classifier. 
In terms of runtime, LogReg runs significantly faster on average. However, for some reason, the best performing model in LogReg took 210s to run. I am not sure whether it is because of my computer or it is supposed to be like that.

**NOTE:** For `sklearns`'s built in classifiers, including svm.SVC, there is a function called `GridSearchCV` which can automatically perform hyperparamater tuning for you. The main advantage of the method (as opposed to writing for-loops) is that it supports parallelization, so it can fit with different parameters at the same time. The function also supports automatic $k$-fold cross-validation (instead of simple train/test split).

You might be interested in using this function in the future. If so, please check out the tutorial in the following lab from previous year: https://github.com/sdrangan/introml/blob/master/unit08_svm/lab_emnist_partial.ipynb.